<a href="https://colab.research.google.com/github/qrak/3commas-TakeProfit-Update/blob/master/breakout2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install nest_asyncio



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ccxt
import ccxt.pro as ccxtpro
import asyncio
import nest_asyncio
from datetime import datetime

nest_asyncio.apply()

exchange = ccxtpro.bitget({
    'enableRateLimit': True,
})

async def get_average_volume(pair, timeframe='1m', num_candles=5):
    try:
        candles = await exchange.fetch_ohlcv(pair, timeframe, limit=num_candles)
    except ccxt.ExchangeError as e:
        raise Exception(f"Error fetching OHLCV data for {pair}: {str(e)}")

    if not candles or len(candles) < num_candles:
        raise Exception(f"Insufficient OHLCV data for {pair}")

    volumes = [c[5] for c in candles]
    return sum(volumes[:-1]) / (len(volumes) - 1), volumes[-1]


async def breakout_strategy():
    try:
        print("Loading markets...")
        symbols = await exchange.load_markets()
        print("Markets loaded.")

        trading_pairs = [symbol for symbol in symbols if symbol.endswith('/USDT')]
        print(f"Trading pairs: {trading_pairs}")

        min_usdt_volume = 1000000
        for pair in trading_pairs:
            try:
                average_volume, current_volume = await get_average_volume(pair)
            except Exception as e:
                print(f"Error getting average volume for {pair}: {e}")
                continue

            if current_volume < min_usdt_volume:
                continue  # Skip the pair if its current volume is below the minimum threshold

            ticker = await exchange.fetch_ticker(pair)
            last_price = ticker['last']
            prev_close_price = ticker['close']

            volume_spike_threshold = 2
            price_spike_threshold = 1.02

            if current_volume > volume_spike_threshold * average_volume:
                print(f"Volume spike detected for {pair}")
                orderbook = await exchange.fetch_order_book(pair)
                bid_price, bid_volume = orderbook['bids'][0]
                prev_bid_price, prev_bid_volume = orderbook['bids'][1]

                if bid_price > price_spike_threshold * last_price and bid_price > prev_bid_price and bid_volume > prev_bid_volume and last_price > prev_close_price:
                    print(f'{datetime.now()} - Breakout detected for {pair}: {bid_price} USDT')

    except Exception as e:
        print(f'Error: {e}')
        await exchange.close()


async def main():
    try:
        while True:
            async with exchange:
                await breakout_strategy()
                await asyncio.sleep(60)
    except KeyboardInterrupt:
        print("Stopping the script.")
    finally:
        await exchange.close()

loop = asyncio.get_event_loop()
loop.run_until_complete(main())





Loading markets...
Markets loaded.
Trading pairs: ['TRX/USDT', 'LINK/USDT', 'UNI/USDT', 'SUSHI/USDT', 'COMP/USDT', 'AAVE/USDT', 'YFI/USDT', 'DOGE/USDT', 'CHZ/USDT', 'ID/USDT', 'FAST/USDT', 'PRY/USDT', 'ARB/USDT', 'STABLZ/USDT', 'GCAKE/USDT', 'CAPS/USDT', 'TAO/USDT', 'FIS/USDT', 'GRV/USDT', 'TVK/USDT', 'BONE/USDT', 'WOOF/USDT', 'RDNT/USDT', 'GRAIL/USDT', 'AGIX/USDT', 'AI/USDT', 'IMGNAI/USDT', 'ORAI/USDT', 'ALI/USDT', 'PSI/USDT', 'ARBINU/USDT', 'ZYB/USDT', 'CORE/USDT', 'GFT/USDT', 'DPX/USDT', 'PLS/USDT', 'CAI/USDT', 'ONE/USDT', 'JONES/USDT', 'SSV/USDT', 'BFR/USDT', 'WAXL/USDT', 'THENA/USDT', 'IDEX/USDT', 'BRISE/USDT', 'INJ/USDT', 'MDAO/USDT', 'IOTX/USDT', 'CHESS/USDT', 'IQ/USDT', 'THALES/USDT', 'KEY/USDT', 'CTR/USDT', 'KOK/USDT', 'ETHF/USDT', 'HAMI/USDT', 'VELA/USDT', 'ARG/USDT', 'BLUR/USDT', 'EURT/USDT', 'GMX/USDT', 'XAUT/USDT', 'VINU/USDT', 'JPEG/USDT', 'LEVER/USDT', 'RARE/USDT', 'LIT/USDT', 'IGU/USDT', 'LOKA/USDT', 'TRU/USDT', 'LOOM/USDT', 'XEN/USDT', 'SUDO/USDT', 'BBO/USDT', 'MTL/USD